In [19]:
# !pip install -r ../exp_requirements.txt

In [20]:
# !pip uninstall virny -y

In [21]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


# COMPAS Dataset With Random Nulls

## Import dependencies

In [4]:
import os
import copy

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from virny.utils.custom_initializers import create_config_obj, create_models_metrics_dct_from_database_df
from virny.datasets.data_loaders import CompasDataset
from source.user_interfaces.experiment_interface import run_exp_iter_with_models_stress_testing
from source.error_injectors.outliers_injector_v2 import OutliersInjectorV2
from source.utils.custom_initializers import create_experiment_data_loader
from source.utils.db_functions import read_model_metric_dfs_from_db
from source.preprocessing.basic_preprocessing import get_null_imputer_preprocessor

from configs.constants import NUM_METRICS_COMPUTATION_RUNS, EXPERIMENT_SEEDS, TEST_SET_FRACTION
from configs.models_params_for_tuning import get_compas_models_params_for_tuning

## Initialize input variables for the experiment

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'stress_testing_with_cols_imp_outliers'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
COLUMNS_TO_TRANSFORM = []
ROWS_PCT_TO_AFFECT = 0.4
INJECTOR_CONFIG_LST = ['priors_count', 'age', 'juv_other_count', 'juv_fel_count']

In [6]:
data_loader = CompasDataset()
data_loader.X_data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,race,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M,sex
0,25,0.0,-2.340451,1.0,-15.010999,African-American,1,0,0,0,1,1
1,26,0.0,0.000000,0.0,0.000000,Caucasian,1,0,0,1,0,0
2,21,0.0,0.000000,0.0,0.000000,Caucasian,0,0,1,1,0,1
3,29,0.0,0.000000,0.0,6.000000,African-American,1,0,0,0,1,1
4,40,0.0,0.000000,0.0,7.513697,Caucasian,1,0,0,1,0,1


In [7]:
data_loader.full_df.shape

(5278, 13)

In [8]:
data_loader.full_df.isna().sum()

age                        0
juv_fel_count              0
juv_misd_count             0
juv_other_count            0
priors_count               0
age_cat_25 - 45            0
age_cat_Greater than 45    0
age_cat_Less than 25       0
c_charge_degree_F          0
c_charge_degree_M          0
race                       0
sex                        0
recidivism                 0
dtype: int64

### Define a db writer and custom fields to insert into your database

In [9]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [10]:
custom_table_fields_dct = {
    'error_type': 'Random Outliers',
    'error_percentages': 'priors_count,age,juv_other_count,juv_fel_count',
    'preprocessing_technique': 'No',
}

In [11]:
import uuid

custom_table_fields_dct['session_uuid'] = '67b67c69-63b9-4ef1-9b02-dee66853eb46'
# custom_table_fields_dct['session_uuid'] = str(uuid.uuid4())
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  67b67c69-63b9-4ef1-9b02-dee66853eb46


### Create a metrics computation config object

In [12]:
config_yaml_path = 'experiment_config.yaml'
config_yaml_content = \
f"""
dataset_name: COMPAS
bootstrap_fraction: 0.8
n_estimators: 200  # Better to input the higher number of estimators than 100; this is only for this use case example
num_runs: 1
runs_seed_lst: [100]
sensitive_attributes_dct: {{'sex': 0, 'race': 'Caucasian', 'sex&race': None}}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [13]:
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

### Define tuning parameter for models

In [14]:
# def get_models_params_for_tuning(models_tuning_seed):
#     return {
#         # 'DecisionTreeClassifier': {
#         #     'model': DecisionTreeClassifier(random_state=models_tuning_seed),
#         #     'params': {
#         #         "max_depth": [20, 30],
#         #         "min_samples_split" : [0.1],
#         #         "max_features": ['sqrt'],
#         #         "criterion": ["gini", "entropy"]
#         #     }
#         # },
#         'LogisticRegression': {
#             'model': LogisticRegression(random_state=models_tuning_seed),
#             'params': {
#                 'penalty': ['l2'],
#                 'C' : [0.0001, 0.1, 1, 100],
#                 'solver': ['newton-cg', 'lbfgs'],
#                 'max_iter': [250],
#             }
#         },
#         # 'RandomForestClassifier': {
#         #     'model': RandomForestClassifier(random_state=models_tuning_seed),
#         #     'params': {
#         #         "max_depth": [6, 10],
#         #         "min_samples_leaf": [1],
#         #         "n_estimators": [50, 100],
#         #         "max_features": [0.6]
#         #     }
#         # },
#         # 'XGBClassifier': {
#         #     'model': XGBClassifier(random_state=models_tuning_seed, verbosity=0),
#         #     'params': {
#         #         'learning_rate': [0.1],
#         #         'n_estimators': [200],
#         #         'max_depth': [5, 7],
#         #         'lambda':  [10, 100]
#         #     }
#         # },
#         # 'KNeighborsClassifier': {
#         #     'model': KNeighborsClassifier(),
#         #     'params': {
#         #         'n_neighbors' : [5, 7, 9, 11, 13, 15, 25],
#         #         'weights' : ['uniform', 'distance'],
#         #         'metric' : ['minkowski', 'euclidean', 'manhattan']
#         #     }
#         # },
#         # 'MLPClassifier': {
#         #     'model': MLPClassifier(random_state=models_tuning_seed),
#         #     'params': {
#         #         'hidden_layer_sizes':[(100,), (100,100,), (100,50,100,)],
#         #         'activation': ['logistic', 'tanh', 'relu'],
#         #         'solver': ['lbfgs', 'sgd', 'adam'],
#         #         'learning_rate': ['constant', 'invscaling', 'adaptive']
#         #     }
#         # }
#     }

## Run experiments

### Experiment iteration 1

In [36]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_1_20230425__175355.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=ROWS_PCT_TO_AFFECT,
                                    max_num_columns_to_effect=1)
models_params_for_tuning = get_compas_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [37]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        # with_tuning=True,
                                        # tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        mode='column_importance',
                                        verbose=True)

2023-05-03 19:23:24 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'error_percentages': 'priors_count,age,juv_other_count,juv_fel_count',
 'error_type': 'Random Outliers',
 'experiment_iteration': 'Exp_iter_1',
 'injector_config_lst': "['priors_count', 'age', 'juv_other_count', "
                        "'juv_fel_count']",
 'model_init_seed': 100,
 'preprocessing_technique': 'No',
 'session_uuid': 'a7330afe-1a66-46fb-a9c8-168e741cf2fa'}




2023-05-03 19:23:24 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([1639, 4204, 3264,  664, 1039,  640, 3765, 2047, 1575,  995,  852,
            3633, 2081, 5121, 2055, 2053, 2087, 3058,  662, 3826],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([1639, 4204, 3264,  664, 1039,  640, 3765, 2047, 1575,  995,  852,
            3633, 2081, 5121, 2055, 2053, 2087, 3058,  662, 3826],
           dtype='int64')
Creating test sets based on column_importance...
error_injector.columns_to_transform --  ['priors_count']
error_injector.seed --  101
transformed_X_test:
 age                        0
juv_fel_count              0
juv_misd_count             0
juv_other_count            0
priors_count               0
race                       0
age_cat_25 - 45            0
age_cat_Greater than 45    0
age_cat_Less than 25       0
c_charge_degree_F          0
c_charge_degree_M          0
sex                        0
dtype: int64


priors_count: 218


error_injector.colum

2023-05-03 19:23:25 experiment_interface.py INFO    : Models config is loaded from the input file


transformed_X_test:
 age                        0
juv_fel_count              0
juv_misd_count             0
juv_other_count            0
priors_count               0
race                       0
age_cat_25 - 45            0
age_cat_Greater than 45    0
age_cat_Less than 25       0
c_charge_degree_F          0
c_charge_degree_M          0
sex                        0
dtype: int64


juv_fel_count: 216


LogisticRegression:  {'C': 0.01, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 50, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l1', 'random_state': 100, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

### Experiment iteration 2

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_2_20230425__180925.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=ROWS_PCT_TO_AFFECT,
                                    max_num_columns_to_effect=1)
models_params_for_tuning = get_compas_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        # with_tuning=True,
                                        # tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        mode='column_importance',
                                        verbose=True)

### Experiment iteration 3

In [15]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_3_20230425__182629.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=ROWS_PCT_TO_AFFECT,
                                    max_num_columns_to_effect=1)
models_params_for_tuning = get_compas_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [16]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        # with_tuning=True,
                                        # tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        mode='column_importance',
                                        verbose=True)

2023-05-03 14:28:04 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'error_percentages': 'priors_count,age,juv_other_count,juv_fel_count',
 'error_type': 'Random Outliers',
 'experiment_iteration': 'Exp_iter_3',
 'injector_config_lst': "['priors_count', 'age', 'juv_other_count', "
                        "'juv_fel_count']",
 'model_init_seed': 300,
 'preprocessing_technique': 'No',
 'session_uuid': '67b67c69-63b9-4ef1-9b02-dee66853eb46'}




2023-05-03 14:28:04 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([4843,  538, 1416, 5274, 1164, 3910, 2163, 3727, 4024, 1258,  141,
            2550, 2313,  665, 2287, 1846,  688, 4598,  568, 2472],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([4843,  538, 1416, 5274, 1164, 3910, 2163, 3727, 4024, 1258,  141,
            2550, 2313,  665, 2287, 1846,  688, 4598,  568, 2472],
           dtype='int64')
Creating test sets based on column_importance...
error_injector.columns_to_transform --  ['priors_count']
error_injector.seed --  301
transformed_X_test:
 age                        0
juv_fel_count              0
juv_misd_count             0
juv_other_count            0
priors_count               0
race                       0
age_cat_25 - 45            0
age_cat_Greater than 45    0
age_cat_Less than 25       0
c_charge_degree_F          0
c_charge_degree_M          0
sex                        0
dtype: int64


priors_count: 219


error_injector.colum

2023-05-03 14:28:05 experiment_interface.py INFO    : Models config is loaded from the input file


juv_fel_count: 214


LogisticRegression:  {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 50, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 300, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_4_20230425__184149.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=ROWS_PCT_TO_AFFECT,
                                    max_num_columns_to_effect=1)
models_params_for_tuning = get_compas_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        # with_tuning=True,
                                        # tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        mode='column_importance',
                                        verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_COMPAS_exp_iter_5_20230425__185721.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=ROWS_PCT_TO_AFFECT,
                                    max_num_columns_to_effect=1)
models_params_for_tuning = get_compas_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        # with_tuning=True,
                                        # tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        mode='column_importance',
                                        verbose=True)

In [38]:
# client.close()